# Download the archive at this [link](https://drive.google.com/file/d/1ayxegsMonJUd_6_NBs4MTEQ9mR7RJ3As/view?usp=sharing) and unzip it in the current folder. Install all the required packages

In [ ]:
# run this if you are in colab
!gdown --id 1ayxegsMonJUd_6_NBs4MTEQ9mR7RJ3As
!unzip fabric-data.zip
!pip install colorthief

## Import all the required libraries

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam, SGD
from sklearn.model_selection  import StratifiedShuffleSplit
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
from colorthief import ColorThief
import seaborn as sn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

# load other models
from tensorflow.keras.applications.resnet_v2 import ResNet50V2, ResNet152V2
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.applications.densenet import DenseNet169, DenseNet201

In [ ]:
def resize(image, dim=100):
    try:
        im = cv2.imread(image)
        resized = cv2.resize(im, (dim, dim), interpolation = cv2.INTER_AREA)
    except Exception as e:
        print(e)
    return resized

# training info
n_folds=5
BATCH_SIZE=16
SGD = SGD(learning_rate=0.1, momentum=0.001)

In [ ]:
# load dataset info
df = pd.read_csv('dataset.csv')
display(df)

In [ ]:
# dataset folders
data_color= "Dataset"
color_reference_folder='Ref/'
ref_dominant='Ref_dominant'

classi = ["Orange", "White", "Blue", "Cyan", "Yellow", "Magenta", "Black", "Red", "Earth Brown", "Green", "Emerald Green", "Purple"]
# classi=["Arancione", "Bianco", "Blu", "Cyan", "Giallo", "Magenta", "Nero", "Rosso", "Terra", "Verde", "Verde_Smeraldo", "Viola"]

if not os.path.exists(ref_dominant):
    os.makedirs(ref_dominant)

## Create the difference domain dataset

In [ ]:
# get the images for each reference color
for elem in os.listdir(color_reference_folder):
    img = resize(color_reference_folder + "/" + elem)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    color_thief = ColorThief(color_reference_folder+"/"+elem)
    dominant_color = color_thief.get_color(quality=1)

    img_color = img.copy()
    img_color[:,:,0] = dominant_color[0]
    img_color[:,:,1] = dominant_color[1]
    img_color[:,:,2] = dominant_color[2]

    img_color = cv2.cvtColor(img_color, cv2.COLOR_RGB2BGR)

    cv2.imwrite('Ref_dominant/'+ elem,img_color)

In [ ]:
# translate images into the difference domain
dataset = []
for index, row in df.iterrows():
  path_img = row['image_path']
  labels = row['color']
  dizionario={}
  temp=[]
  for elem in os.listdir(ref_dominant):
    img_dominant = cv2.imread(path_img)
    img_ref = cv2.imread(ref_dominant+'/'+elem)
    result = img_dominant - img_ref
    result = (result/255).astype(np.float16)
    temp.append(result[:,:,0])
    temp.append(result[:,:,1])
    temp.append(result[:,:,2])
  temp.append(labels)
  dataset.append(temp)

In [ ]:
# encode classes
le = LabelEncoder()
le.fit(classi)

x=[x[0:36] for x in dataset]
x = np.array(x)
x= np.moveaxis(x,1,3)

y=[elem[36] for elem in dataset ]
y = le.transform(y)
y=np.array(y)
y = to_categorical(y)

In [ ]:
# split and hold a part of dataset for testing
X_hold, X_test, y_hold, y_test = train_test_split(x, y, test_size=0.2, stratify=y)

In [ ]:
def cnn_difference():
  model = Sequential()

  model.add(Conv2D(32, 3, 3, input_shape = (100, 100, 36), activation = 'relu'))

  model.add(MaxPooling2D(pool_size = (2, 2)))

  model.add(Conv2D(32, 3, 3, activation = 'relu'))
  model.add(MaxPooling2D(pool_size = (2, 2)))

  model.add(Flatten())

  model.add(Dense(128, activation = 'relu', kernel_regularizer='l2'))
  model.add(Dense(12, activation = 'softmax'))


  model.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])
  return model

## ResNet50v2

In [ ]:
fold = 0
sss = StratifiedShuffleSplit(n_splits=n_folds, random_state=15 , test_size=0.2)
acc = [0,0]
for index_train, index_test in sss.split(x, y):
  fold =fold+1
  print(str(fold)+"/"+str(n_folds))
  X_hold, X_test = (x[index_train],x[index_test])
  y_hold, y_test = (y[index_train], y[index_test])


  X_train, X_val, y_train, y_val = train_test_split(X_hold, y_hold, test_size=0.2, stratify=y_hold)


  resnet50 = ResNet50V2(include_top=True, 
                        weights=None, 
                        input_tensor=None,
                        input_shape = (100, 100, 36), 
                        pooling=None, 
                        classes=12,
                        classifier_activation='softmax')
  
  resnet50.compile(loss='categorical_crossentropy', 
                   optimizer=SGD,
                   metrics=['accuracy'])
  
  # Defining epoch step sizes based on number of images divided by batch size
  step_train = len(X_train)//BATCH_SIZE
  step_valid = len(X_val)//BATCH_SIZE
  step_test = (len(X_test))//BATCH_SIZE
  
  model_checkpoint_callback = ModelCheckpoint( filepath=f'model_resnet50_fold{fold}.h5',
                                              save_weights_only=False,
                                              monitor='val_loss',
                                              mode='max',
                                              save_best_only=True)

  early_stopping_callback = EarlyStopping(monitor='val_loss', patience=4)

  #Training fold
  history=resnet50.fit(X_train, y_train, batch_size=BATCH_SIZE, validation_data= (X_val, y_val),validation_steps=step_valid, epochs=50, steps_per_epoch=step_train, callbacks=[early_stopping_callback])

    #Predict
  y_pred = resnet50.predict(X_test)
  y_pred = np.argmax(y_pred, axis=1)

  #Evaluation model
  print("Evaluating fold "+str(fold))

  y_true = np.argmax(y_test , axis=1)

  acc_temp = accuracy_score(y_true,y_pred)
  if acc[0] < acc_temp:
    acc[0] = acc_temp
    acc[1] = fold


  report = classification_report(y_true, y_pred ,target_names=classi , output_dict=True)
  print(classification_report(y_true, y_pred ,target_names=classi))
  df_report = pd.DataFrame(report).T
  df_report.to_csv(f'resnet50_report_fold{fold}.csv')

  array = confusion_matrix(y_true, y_pred )
  cmatrix = pd.DataFrame(array, index = [i for i in classi],
                  columns = [i for i in classi])
  plt.figure(figsize = (10,7))
  z = sn.heatmap(cmatrix, annot=True)

  print(f"Best model is model_fold{acc[1]}.h5")

## ResNet152V2

In [ ]:
fold = 0
sss = StratifiedShuffleSplit(n_splits=n_folds, random_state=15 , test_size=0.2)
acc = [0,0]
for index_train, index_test in sss.split(x, y):
  fold =fold+1
  print(str(fold)+"/"+str(n_folds))
  X_hold, X_test = (x[index_train],x[index_test])
  y_hold, y_test = (y[index_train], y[index_test])


  X_train, X_val, y_train, y_val = train_test_split(X_hold, y_hold, test_size=0.2, stratify=y_hold)


  resnet152 = ResNet152V2(include_top=True, 
                        weights=None, 
                        input_tensor=None,
                        input_shape = (100, 100, 36), 
                        pooling=None, 
                        classes=12,
                        classifier_activation='softmax')
  
  resnet152.compile(loss='categorical_crossentropy', 
                   optimizer=SGD,
                   metrics=['accuracy'])
  
  # Defining epoch step sizes based on number of images divided by batch size
  step_train = len(X_train)//BATCH_SIZE
  step_valid = len(X_val)//BATCH_SIZE
  step_test = (len(X_test))//BATCH_SIZE
  
  model_checkpoint_callback = ModelCheckpoint( filepath=f'model_resnet152_fold{fold}.h5',
                                              save_weights_only=False,
                                              monitor='val_loss',
                                              mode='max',
                                              save_best_only=True)

  early_stopping_callback = EarlyStopping(monitor='val_loss', patience=4)

  #Training fold
  history=resnet152.fit(X_train, y_train, batch_size=BATCH_SIZE, validation_data= (X_val, y_val),validation_steps=step_valid, epochs=50, steps_per_epoch=step_train, callbacks=[early_stopping_callback])

    #Predict
  y_pred = resnet152.predict(X_test)
  y_pred = np.argmax(y_pred, axis=1)

  #Evaluation model
  print("Evaluating fold "+str(fold))

  y_true = np.argmax(y_test , axis=1)

  acc_temp = accuracy_score(y_true,y_pred)
  if acc[0] < acc_temp:
    acc[0] = acc_temp
    acc[1] = fold


  report = classification_report(y_true, y_pred ,target_names=classi , output_dict=True)
  print(classification_report(y_true, y_pred ,target_names=classi))
  df_report = pd.DataFrame(report).T
  df_report.to_csv(f'resnet152_report_fold{fold}.csv')

  array = confusion_matrix(y_true, y_pred )
  cmatrix = pd.DataFrame(array, index = [i for i in classi],
                  columns = [i for i in classi])
  plt.figure(figsize = (10,7))
  z = sn.heatmap(cmatrix, annot=True)

  print(f"Best model is model_fold{acc[1]}.h5")

## InceptionResNetV2

In [ ]:
fold = 0
sss = StratifiedShuffleSplit(n_splits=n_folds, random_state=15 , test_size=0.2)
acc = [0,0]
for index_train, index_test in sss.split(x, y):
  fold =fold+1
  print(str(fold)+"/"+str(n_folds))
  X_hold, X_test = (x[index_train],x[index_test])
  y_hold, y_test = (y[index_train], y[index_test])


  X_train, X_val, y_train, y_val = train_test_split(X_hold, y_hold, test_size=0.2, stratify=y_hold)


  inception = InceptionResNetV2(include_top=True, 
                        weights=None, 
                        input_tensor=None,
                        input_shape = (100, 100, 36), 
                        pooling=None, 
                        classes=12,
                        classifier_activation='softmax')
  
  inception.compile(loss='categorical_crossentropy', 
                   optimizer=SGD,
                   metrics=['accuracy'])
  
  # Defining epoch step sizes based on number of images divided by batch size
  step_train = len(X_train)//BATCH_SIZE
  step_valid = len(X_val)//BATCH_SIZE
  step_test = (len(X_test))//BATCH_SIZE
  
  model_checkpoint_callback = ModelCheckpoint( filepath=f'model_inception_fold{fold}.h5',
                                              save_weights_only=False,
                                              monitor='val_loss',
                                              mode='max',
                                              save_best_only=True)

  early_stopping_callback = EarlyStopping(monitor='val_loss', patience=4)

  #Training fold
  history=inception.fit(X_train, y_train, batch_size=BATCH_SIZE, validation_data= (X_val, y_val),validation_steps=step_valid, epochs=50, steps_per_epoch=step_train, callbacks=[early_stopping_callback])

    #Predict
  y_pred = inception.predict(X_test)
  y_pred = np.argmax(y_pred, axis=1)

  #Evaluation model
  print("Evaluating fold "+str(fold))

  y_true = np.argmax(y_test , axis=1)

  acc_temp = accuracy_score(y_true,y_pred)
  if acc[0] < acc_temp:
    acc[0] = acc_temp
    acc[1] = fold


  report = classification_report(y_true, y_pred ,target_names=classi , output_dict=True)
  print(classification_report(y_true, y_pred ,target_names=classi))
  df_report = pd.DataFrame(report).T
  df_report.to_csv(f'inception_report_fold{fold}.csv')

  array = confusion_matrix(y_true, y_pred )
  cmatrix = pd.DataFrame(array, index = [i for i in classi],
                  columns = [i for i in classi])
  plt.figure(figsize = (10,7))
  z = sn.heatmap(cmatrix, annot=True)

  print(f"Best model is model_fold{acc[1]}.h5")

## DenseNet169

In [ ]:
fold = 0
sss = StratifiedShuffleSplit(n_splits=n_folds, random_state=15 , test_size=0.2)
acc = [0,0]
for index_train, index_test in sss.split(x, y):
  fold =fold+1
  print(str(fold)+"/"+str(n_folds))
  X_hold, X_test = (x[index_train],x[index_test])
  y_hold, y_test = (y[index_train], y[index_test])


  X_train, X_val, y_train, y_val = train_test_split(X_hold, y_hold, test_size=0.2, stratify=y_hold)


  densenet169 = DenseNet169(include_top=True, 
                        weights=None, 
                        input_tensor=None,
                        input_shape = (100, 100, 36), 
                        pooling=None, 
                        classes=12)
  
  densenet169.compile(loss='categorical_crossentropy', 
                   optimizer=SGD,
                   metrics=['accuracy'])
  
  # Defining epoch step sizes based on number of images divided by batch size
  step_train = len(X_train)//BATCH_SIZE
  step_valid = len(X_val)//BATCH_SIZE
  step_test = (len(X_test))//BATCH_SIZE
  
  model_checkpoint_callback = ModelCheckpoint( filepath=f'model_densenet169_fold{fold}.h5',
                                              save_weights_only=False,
                                              monitor='val_loss',
                                              mode='max',
                                              save_best_only=True)

  early_stopping_callback = EarlyStopping(monitor='val_loss', patience=4)

  #Training fold
  history=densenet169.fit(X_train, y_train, batch_size=BATCH_SIZE, validation_data= (X_val, y_val),validation_steps=step_valid, epochs=50, steps_per_epoch=step_train, callbacks=[early_stopping_callback])

    #Predict
  y_pred = densenet169.predict(X_test)
  y_pred = np.argmax(y_pred, axis=1)

  #Evaluation model
  print("Evaluating fold "+str(fold))

  y_true = np.argmax(y_test , axis=1)

  acc_temp = accuracy_score(y_true,y_pred)
  if acc[0] < acc_temp:
    acc[0] = acc_temp
    acc[1] = fold


  report = classification_report(y_true, y_pred ,target_names=classi , output_dict=True)
  print(classification_report(y_true, y_pred ,target_names=classi))
  df_report = pd.DataFrame(report).T
  df_report.to_csv(f'densenet169_report_fold{fold}.csv')

  array = confusion_matrix(y_true, y_pred )
  cmatrix = pd.DataFrame(array, index = [i for i in classi],
                  columns = [i for i in classi])
  plt.figure(figsize = (10,7))
  z = sn.heatmap(cmatrix, annot=True)

  print(f"Best model is model_fold{acc[1]}.h5")

## DenseNet201

In [ ]:
fold = 0
sss = StratifiedShuffleSplit(n_splits=n_folds, random_state=15 , test_size=0.2)
acc = [0,0]
for index_train, index_test in sss.split(x, y):
  fold =fold+1
  print(str(fold)+"/"+str(n_folds))
  X_hold, X_test = (x[index_train],x[index_test])
  y_hold, y_test = (y[index_train], y[index_test])


  X_train, X_val, y_train, y_val = train_test_split(X_hold, y_hold, test_size=0.2, stratify=y_hold)


  densenet201 = DenseNet201(include_top=True, 
                        weights=None, 
                        input_tensor=None,
                        input_shape = (100, 100, 36), 
                        pooling=None, 
                        classes=12)
  
  densenet201.compile(loss='categorical_crossentropy', 
                   optimizer=SGD,
                   metrics=['accuracy'])
  
  # Defining epoch step sizes based on number of images divided by batch size
  step_train = len(X_train)//BATCH_SIZE
  step_valid = len(X_val)//BATCH_SIZE
  step_test = (len(X_test))//BATCH_SIZE
  
  model_checkpoint_callback = ModelCheckpoint( filepath=f'model_densenet201_fold{fold}.h5',
                                              save_weights_only=False,
                                              monitor='val_loss',
                                              mode='max',
                                              save_best_only=True)

  early_stopping_callback = EarlyStopping(monitor='val_loss', patience=4)

  #Training fold
  history=densenet201.fit(X_train, y_train, batch_size=BATCH_SIZE, validation_data= (X_val, y_val),validation_steps=step_valid, epochs=50, steps_per_epoch=step_train, callbacks=[early_stopping_callback])

    #Predict
  y_pred = densenet201.predict(X_test)
  y_pred = np.argmax(y_pred, axis=1)

  #Evaluation model
  print("Evaluating fold "+str(fold))

  y_true = np.argmax(y_test , axis=1)

  acc_temp = accuracy_score(y_true,y_pred)
  if acc[0] < acc_temp:
    acc[0] = acc_temp
    acc[1] = fold


  report = classification_report(y_true, y_pred ,target_names=classi , output_dict=True)
  print(classification_report(y_true, y_pred ,target_names=classi))
  df_report = pd.DataFrame(report).T
  df_report.to_csv(f'densenet201_report_fold{fold}.csv')

  array = confusion_matrix(y_true, y_pred )
  cmatrix = pd.DataFrame(array, index = [i for i in classi],
                  columns = [i for i in classi])
  plt.figure(figsize = (10,7))
  z = sn.heatmap(cmatrix, annot=True)

  print(f"Best model is model_fold{acc[1]}.h5")